# Fazendo as inferências nas imagens

In [1]:
# Tentar rodar em paralelo - OK
# Abrir a imagem - OK
# gerando GrayScale - OK
# aplicar Scaler - OK
# gerar Filtro de Sobel - OK
# aplicar Scaler ao Sobel - OK
# gerar LBP - OK
# aplicar Scaler no LBP - OK
# separar as quadriculas - OK
# aplicar PCA - OK
# Inferir se é faixa de pedestre - OK
# salvar imagem 1bit por SCM

## DEpois gerando dados

# agrupar em uma única imagem para toda a ciade
# agrupar dbScan
# gerar polígonos
# gerar linhas

In [2]:
import multiprocessing as mp
import geopandas as gpd
from sklearnex import patch_sklearn
patch_sklearn()
import rasterio
from rasterio import features
import numpy as np
import glob
import joblib
from skimage import filters
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import geopandas as gpd
from shapely.geometry import box
from sklearn.svm import SVC


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
gdf_scms = gpd.read_file('downloads/SIRGAS_SHP_quadriculaortofoto2017.zip!SIRGAS_SHP_quadriculaortofoto2017/SIRGAS_SHP_quadriculaortofoto2017.shp')

In [4]:
n_cpu = mp.cpu_count() - 1

In [5]:
sample =  gdf_scms[2:17]['qo2010_cod'].to_list()

In [6]:
len(sample)

15

In [7]:
sample

['3335-313',
 '2346-434',
 '3336-313',
 '3335-434',
 '3335-433',
 '3335-424',
 '3335-423',
 '3335-414',
 '3335-413',
 '3335-334',
 '3335-333',
 '3335-324',
 '3335-323',
 '3335-314',
 '2346-423']

In [8]:
radius = 3
n_points = 8 * radius
tamanho_janela = 24 # tamanho em pixels
sobreposicao_de_janelas = 12 # sobreposição das janelas em pixels

In [9]:
gdf_quadras = gpd.read_file('zip://downloads/SIRGAS_SHP_quadraMDSF.zip!/SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp')

In [10]:
grayscale_scaler = joblib.load('pre-process/grayscale.scaler.save')
sobel_scaler = joblib.load('pre-process/sobel.scaler.save') 
lbp_scaler = joblib.load('pre-process/lbp.scaler.save')

In [11]:
pca = joblib.load('resultados/modelos-treinados/PCA-faixas-pedestre.pkl')

In [12]:
clf = joblib.load('resultados/modelos-treinados/SVC-faixas-pedestre.pkl')

In [13]:
def faixas(scm):
    file = glob.glob(f'../ortofotos-2017/RGB-2017/*{scm}*.jp2')
    dataset = rasterio.open(file[0])
    data, location = [], []

    grayscale = 0.2125 * dataset.read(1) + \
            0.7154 * dataset.read(2) + \
            0.0721 * dataset.read(3)
    grayscale_uint = grayscale.astype('uint8')

    grayscale = grayscale_scaler.transform(grayscale_uint.reshape(-1, 1)).reshape(grayscale_uint.shape).astype('float32')

    with rasterio.open(
        f'resultados/raster/grayscale-{scm}.jp2', 'w',
        driver='GTiff',
        dtype=rasterio.float32,
        count=1,
        width=dataset.width,
        height=dataset.height,
        transform=dataset.transform,
        crs = rasterio.crs.CRS.from_string('EPSG:31983')) as dst:
        dst.write(grayscale, indexes=1)

    sobel = filters.sobel(grayscale_uint).astype('float32')
    sobel = sobel_scaler.transform(sobel.reshape(-1, 1)).reshape(sobel.shape)

    # with rasterio.open(
    #     f'resultados/raster/sobel-{scm}.jp2', 'w',
    #     driver='GTiff',
    #     dtype=rasterio.float32,
    #     count=1,
    #     width=dataset.width,
    #     height=dataset.height,
    #     transform=dataset.transform,
    #     crs = rasterio.crs.CRS.from_string('EPSG:31983')) as dst:
    #     dst.write(sobel, indexes=1)

    lbp = local_binary_pattern(grayscale_uint, n_points, radius).astype('float32')
    lbp = lbp_scaler.transform(lbp.reshape(-1, 1)).reshape(lbp.shape)

    # with rasterio.open(
    #     f'resultados/raster/lbp-{scm}.jp2', 'w',
    #     driver='GTiff',
    #     dtype=rasterio.float32,
    #     count=1,
    #     width=dataset.width,
    #     height=dataset.height,
    #     transform=dataset.transform,
    #     crs = rasterio.crs.CRS.from_string('EPSG:31983')) as dst:
    #     dst.write(lbp, indexes=1)

    quadras_scm = gpd.clip(gdf_quadras, gpd.GeoSeries(box(*dataset.bounds))).unary_union

    quadras = features.rasterize(
        ((g, 255) for g in quadras_scm),
        out_shape=dataset.shape,
        transform=dataset.transform)

    for i in np.arange(0, grayscale.shape[0] - tamanho_janela, tamanho_janela - sobreposicao_de_janelas):
        for j in np.arange(0, grayscale.shape[1] - tamanho_janela, tamanho_janela - sobreposicao_de_janelas):
            # Se mais de 98% dos pixels estiverem fora da quadra
            if np.count_nonzero(quadras[i:i+tamanho_janela, j:j+tamanho_janela] == 255) < 0.02 * (tamanho_janela ** 2):
                # print(i, j)
                img_gray = grayscale[i:i+tamanho_janela, j:j+tamanho_janela]
                img_sobel = sobel[i:i+tamanho_janela, j:j+tamanho_janela]
                img_lbp = lbp[i:i+tamanho_janela, j:j+tamanho_janela]
                
                img = np.concatenate([
                    img_gray,
                    img_sobel,
                    img_lbp
                ]).reshape(1, 3 * (tamanho_janela ** 2))
        
                data.append(img.reshape(1, 3 * (tamanho_janela ** 2)))
                location.append([i, j])

    np_data = np.array(data).squeeze()
    np_data = pca.transform(np_data)

    y = clf.predict(np_data)

    matrix = np.zeros([dataset.height, dataset.width], dtype='uint8')

    for i, l in enumerate(location):
        matrix[l[0]:l[0] + tamanho_janela, l[1]:l[1] + tamanho_janela] = y[i]

    profile = dataset.profile
    profile['count'] = 1
    
    with rasterio.open(f'resultados/raster/faixas-{scm}.jp2', 'w', nbits=1, **profile) as dst:
        dst.write(matrix, indexes=1)

    return y, location, np.array(data).squeeze()

In [14]:
# y, location, data = faixas('3313-432')
y, location, data = faixas('3314-414')

In [15]:
# with mp.Pool(processes=n_cpu) as p:
#     p.map(faixas, sample)

In [16]:
# seila = np.load('pre-process/data/data-3313-432.npy')

In [17]:
# data.shape, seila.shape

In [18]:
# seila[:, 2*tamanho_janela**2:] - data[:, 2 * tamanho_janela ** 2:]

In [19]:
# seila.dtype

In [20]:
# seila.shape